In [6]:
!pip install umap-learn

In [1]:
import umap
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
import torch
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm


/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [79]:
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
stopwords = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    #print(words)
    res = list()
    for word in words:
        if (word not in stopwords and len(word) > 1):
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    text = " ".join(res)
    return text

def text_cleaner(input_text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_url= re.sub(r'([a-z]{2}\d+[a-z]{2})',' ',rem_url)
    rem_url = re.sub(r'!+','.', rem_url)
    rem_num = re.sub('[0-9]+', '', rem_url)
    text = re.sub(r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+.,!]+", " ", rem_num)
    text = re.sub("!",".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]",'', text)
    emoji_pattern=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    text = text.lower()
    return text


def dataset_preprocessor_vectorizers(data, w_count_to_drop=3):
    data_frame = data.copy()

    indexes_to_drop = []
    # Дропаем дубли
    data_frame = data_frame.drop_duplicates(subset=['Текст инцидента']).reset_index(drop=True)

    # Regex для групп
    p = re.compile("\[.+\|.+\]")

    for index, row in tqdm(data_frame.iterrows()):

        # Очистка текста
        data_frame.loc[index, "Текст инцидента"] = text_cleaner(data_frame.loc[index, "Текст инцидента"])

        # Отсекаем сообщения из w_count_to_drop слов
        if len(row["Текст инцидента"].split(" ")) < w_count_to_drop and len(row["Текст инцидента"].split(".")) < w_count_to_drop:
            indexes_to_drop.append(index)

    # Дропаем индексы < w_count_to_drop слов
    #print(indexes_to_drop)
    data_frame = data_frame.drop(index=indexes_to_drop).reset_index(drop=True)

    # Лемматизация и удаление стоп слов
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = lemmatize(row["Текст инцидента"])
    return data_frame
train = dataset_preprocessor_vectorizers(train)
test = dataset_preprocessor_vectorizers(test)

In [95]:
train = pd.read_csv("datasets/sbert_large_mt_nlu_ru_train_mean.csv")
test = pd.read_csv("datasets/sbert_large_mt_nlu_ru_test_mean.csv")
embeddings_columns = [str(i) for i in range(1024)]

In [82]:
train = dataset_preprocessor_vectorizers(train)
test = dataset_preprocessor_vectorizers(test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [43]:
train[["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]]

,Текст инцидента,Группа тем,Исполнитель,Тема
0,краснокамск новостикраснокамск объявлениякрасн...,Мусор/Свалки/ТКО,АО ПРО ТКО,★ Уборка/Вывоз мусора
1,ещё два соцзащита продублировать деньга выплат...,Социальное обслуживание и защита,Министерство социального развития ПК,Дети и многодетные семьи
2,добрый день коминтерн комната иметься документ...,ЖКХ,ИГЖН ПК,Жалобы на управляющие компании
3,дом сосулька пермский край,Социальное обслуживание и защита,Город Пермь,Аварийное жилье/переселение
4,пермь просить позортесь отремонтировать остано...,Общественный транспорт,Город Пермь,Содержание остановок
...,...,...,...,...
15087,класс вопрос построить спортивный комплекс рай...,Физическая культура и спорт,Город Пермь,Строительство спортивной инфраструктуры
15088,здравствуйте кунгур привиться завтра?,Коронавирус,Министерство здравоохранения,Порядок и пункты вакцинации
15089,женщина сделать кесарев пермь день выписать до...,Здравоохранение/Медицина,Министерство здравоохранения,★ Оказание медицинской помощи не в полном объе...
15090,вопросовчиновникамный набережная мотовилихинск...,Дороги,Город Пермь,Необходима установка и замена дорожных ограждений


In [96]:
import pickle
class Model_tfidf_emb_umap:
    def __init__(self, vectorizer_name, vectorizer, max_features=None, emb_length=1024, emb_umap_size=100, vectorizer_umap_size=500):
        self.max_features = max_features
        self.vectorizer = vectorizer(max_features=self.max_features)
        self.vectorizer_name = vectorizer_name
        self.emb_length = emb_length
    
    def create_umap(self, emb_names, train, test, cols_for_umap, n_components=100):
        column_names = [emb_names+f"_{i}" for i in range(n_components)]
        ump = umap.UMAP(n_components=n_components, random_state=42)
        train_data_frame = train.copy()
        test_data_frame = test.copy()
        train_emb = ump.fit_transform(train_data_frame[cols_for_umap])
        test_emb = ump.transform(test_data_frame[cols_for_umap])
        
        return pd.DataFrame(train_emb, columns=column_names) , pd.DataFrame(test_emb, columns=column_names)
    
    def create_tfidf(self, train, test):
      tf = self.vectorizer.fit_transform(train)
      with open(f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pk', 'wb') as fin:
        pickle.dump(self.vectorizer, fin)
      vocab = self.vectorizer.vocabulary_
      train_tf, test_tf = self.create_umap("tfidf", pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys())),
                             pd.DataFrame(self.vectorizer.transform(test).toarray(), columns=sorted(vocab.keys())),
                                           cols_for_umap=sorted(vocab.keys()), 
                                           n_components=350)
      del tf
      return train_tf, test_tf


    def create_datasets(self, train, test):
        train_, test_ = train.copy(), test.copy()
        
        embeddings_columns = [str(i) for i in range(self.emb_length)]
        train_emb, test_emb = self.create_umap("bert_emb", 
                                          train_[embeddings_columns] ,
                                          test_[embeddings_columns], 
                                          embeddings_columns, 100)
        train_ = train_[["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]]
        test_ = test_[["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]]
        
        train_ = pd.concat([train_, train_emb], axis=1)
        test_ = pd.concat([test_, test_emb], axis=1)
        
        train_tf_idf, test_tf_idf = self.create_tfidf(train_["Текст инцидента"].to_list(), test_["Текст инцидента"].to_list())
        
       
        
        train_ = pd.concat([train_, train_tf_idf], axis=1)
        test_ = pd.concat([test_, test_tf_idf], axis=1)
        
        del train_tf_idf
        del test_tf_idf
        #train_.to_csv(f"datasets/{self.vectorizer_name}_train_{self.max_features}.csv", index=False)
        #test_.to_csv(f"datasets/{self.vectorizer_name}_test_{self.max_features}.csv", index=False)

        return train_, test_

In [97]:
mtf = Model_tfidf_emb_umap("tfidf", TfidfVectorizer, max_features=5, emb_length=1024) 
train, test = mtf.create_datasets(train, test)

KeyboardInterrupt: 

In [88]:
train.to_csv(f"datasets/sbert100_tfidf350_umap_train.csv", index=False)
test.to_csv(f"datasets/sbert100_tfidf350_umap_test.csv", index=False)

In [89]:
mlflow.set_experiment('sbert100_tfidf350_umap_class_weight')

2023/11/24 21:04:50 INFO mlflow.tracking.fluent: Experiment with name 'sbert100_tfidf350_umap_class_weight' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///workspace/mlruns/470649283789315185', creation_time=1700859890086, experiment_id='470649283789315185', last_update_time=1700859890086, lifecycle_stage='active', name='sbert100_tfidf350_umap_class_weight', tags={}>

In [90]:
param_grid = {
    'iterations': [100, 200, 400, 500]
}
grid = list(ParameterGrid(param_grid))

In [91]:
#for model_ in models:
    #model_name, vectorizer, max_features = model_
    #model = Model_tfidf(model_name, vectorizer, max_features)

train_, test_ = train, test

for params in grid:
    try:
        with mlflow.start_run(nested=True):
            catboost_model = CatBoostClassifier(**params, loss_function='MultiClassOneVsAll', class_weights=class_weights, verbose=0, random_seed=42)
            catboost_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), train_["Группа тем"])

            predictions = catboost_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))

            accuracy = accuracy_score(test_["Группа тем"], predictions)
            f1 = f1_score(test_["Группа тем"], predictions, average='weighted')
            report = classification_report(test_["Группа тем"], predictions, output_dict=True)
            report_text =  classification_report(test_["Группа тем"], predictions)


            mlflow.log_metric("report_accuracy", report['accuracy'])
            mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
            mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
            mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
            mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
            mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
            mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])

            mlflow.log_text(report_text, "classification_report.txt")

            
            mlflow.set_tag("max_features", 15000)
            mlflow.set_tag("dataset_name", "sbert100_tfidf350_umap")
            mlflow.set_tag("model_name", "catboost")
            mlflow.catboost.log_model(catboost_model, "model")
            mlflow.log_params(params)
            mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
    except Exception as e:
        error_name = type(e).name
        print(f"Caught an error: {error_name}")
        print(f"EXCEPTION: {e}")
        print(sys.exc_info()[0])
        traceback.print_exc(file=sys.stdout)